#### 참고자료
- https://pytorch-geometric.readthedocs.io/en/latest/notes/create_gnn.html#the-messagepassing-base-class
- https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.MessagePassing


In [1]:
import torch

try:
    import torch_geometric
except ModuleNotFoundError:
    # Installing torch geometric packages with specific CUDA+PyTorch version.
    # See https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html for details
    TORCH = torch.__version__.split('+')[0]
    CUDA = 'cu' + torch.version.cuda.replace('.','')

    !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-geometric

c:\Users\chang\Anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### MessagePassing
- Pytorch Geometric 라이브러리에서 제공하는 message passing layer를 생성하는 Base Class 이다.
- args 5개 사용
    - (1) aggr : 집계방식지정 --> 선택지 : add, sum, mean, min, max, mul
    - (2) aggr_kwargs : 뭔지 모르겠음
    - (3) flow : message passing의 flow 방향 --> 선택지 : source_to_target(default), target_to_source
    - (4) node_dim : propagate 되는 축 방향 --> 선택지 : int형 기입 (default : -2)
    - (5) decomposed_layers : feature를 분해할 레이어 개수 (메모리 사용과 속도 개선 가능) --> 선택지 : int형 기입 (보통 2를 사용)

In [7]:
import torch
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

# MessagePassing 을 상속받은 GCNConv Class
class GCNConv(MessagePassing): 
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.linear = Linear(in_channels, out_channels, bias=False)
        self.bias = Parameter(torch.Tensor(out_channels))
        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self, x, edge_index):
        # x : [N, in_channels]
        # edge_index : [2, E]

        # Step 1: adjacency matrix에 self_loop 추가
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: node의 feature matrix를 선형변환
        x = self.linear(x)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)  # 주어진 1차원 벡터의 degree(몇개인지) 를 구함
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: message propagate 시작
        out = self.propagate(edge_index, x=x, norm=norm)

        # Step 6: bias 추가
        out += self.bias

        return out

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j

In [ ]:
conv = GCNConv(16, 32)
x = conv(x, edge_index)